In [22]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import math
import random
from typing import Callable
import itertools
from sklearn.svm import SVC

# Описание алгоритмов

## SGD (Stochastic gradient descent)
Cтохастический градиентный спуск – метод поиска минимума для функций вида $f = f1 + f2 + ... +  fn$.    
В отличие от обычного градиентного спуска, в SGD градиент будет браться от случайного слагаемого функции, а не от ее всей.    
Общая задача:
\begin{align}
        \sum_{i=1}^n L_i(w) + λ \cdot R(w) → min.
    \end{align}
где   
1. $w$ - параметры, которые мы обучаем;    
2. $L$ - lost function - ошибка нашей системы на конкретном экземпляре;       
3. сумма идет по всей базе данных, на которой мы обучаем. База данных выглядит следующим образом $\{x(i), y(i)\}$, где $x(i)$ - запрос, $y(i)$ - правильный ответ;    
4. $R$ - регуляризация, какие-то дополнительные ограничения (например, норма вектора, если мы хотим минимизировать параметры).

### Алгоритм:
Фиксируем 2 гипер-параметра: $h$ - темп обучения, $λ$ - темп забывания.
Считаем:
1. $w(0)$ - начальное значение параметров;   
2. $Q = \frac{1}{m} \cdot \sum_{k=1}^m L_{j_k}(w^{(0)}) $ - средняя ошибка в начале по какой-то выборке (возможно по всей)   

На итерации $t$ эпохи обучения $e$ (эпоха заканчивается, когда мы обработали все $L_i$):
1. выбираем новый уникальный $i$;
2. вычисляем $grad(L_i(w^{(t)}))$, обновляем параметры
\begin{align}
        w^{(t + 1)} = w^{(t)} - h \cdot grad(L_i(w^{(t)}))
    \end{align}
3. вычисляем $\epsilon_i = L_i(w^{(t)})$, обновляем значение ошибки
\begin{align}
       Q = λ \cdot ϵ_i + (1 - λ) \cdot Q
    \end{align}
Если $Q$ сходится, завершаем выполнение.


В случае **mini-batch SGD** на одной итерации обрабатывается множество $\{ L_i \}$, размера batchSize.

**Затухание** – **attenuationParameter** –  служит для остановки обучения в хорошем месте и избежания колебаний – ситуации, которая может возникнуть, когда из-за слишком высокого постоянного темпа обучения происходит перепрыгивание через минимум.
О функциях изменения шага (learning rate scheduling):
1. шаг может быть постоянным на протяжении всего обучения;
2. основанный на времени темп обучения
\begin{align}
       h_{n + 1} = h_n / (1 + d \cdot n)
\end{align}
где $h$ - темп обучения, $d$ - параметр затухания, $n$ - шаг итерации;
3. экспоненциальный темп обучения
\begin{align}
       h_{n} = h_0 \cdot e^{- dn}
\end{align}
\begin{align}
       h_{n + 1} = h_n \cdot e^{- d}
\end{align}


# Реализация алгоритмов

В реализации использовался метод наименьших квадратов.
То есть функция потерь (целевая функция) выглядит так:
\begin{align}
       L_i = \sum_{i} L_{(i)} = \sum_{i} (y_{(i)} - \widehat{y_{(i)}})^2 = \sum_{i} (y_{(i)} - w \times x_{(i)})^2
\end{align}
где $w$ - параметры, которые обучаем, $\times$ - скалярное произведение векторов.    
А градиент $L_{(i)}$  так:
\begin{align}
       grad(L_{(i)}) = 2 \cdot x_{(i)} \cdot (w \times x_{(i)} - y_{(i)})
\end{align}


Функция data_fabric по значениям data_size(int) и coefficients(vector) генерирует линейную регрессию (которую далее мы пытаемся восстановить) –
множество размера data_size и вида $\{ x(i), y(i) \}$, где $y_{(i)}$ – значение,  $x_{(i)}$ – вектор:
\begin{align}
       y_{(i)} = \sum_{k = 0}^{len(coefficients)} coefficients_k \cdot randomNumber =  \sum_{k = 0}^{len(coefficients)} x_{(i)_k}
\end{align}


In [23]:
# Функции для генерации линейной регрессии вида: y = coef1 * (x1 ** deg1) + ... coefn * (xn ** degn)

def generate(coefficients: np.array,
             degrees: np.array):
    n = len(coefficients)
    x = np.random.rand(n)
    y = sum((pow(x[i], degrees[i]) * coefficients[i]) for i in range(n))
    return [x, y]

def data_fabric(data_size: int,
                coefficients: np.array,
                degrees: np.array):
    x = np.array([])
    y = np.array([])
    for i in range(data_size):
        new_data = generate(coefficients, degrees)
        x = np.append(x, new_data[0])
        y = np.append(y, new_data[1])
    x = x.reshape(data_size, len(coefficients))
    return [x, y]

In [24]:
# Реализация SGD - основное задание

def stochastic_gradient_descent(X: np.array,
                                Y: np.array,
                                forgetting_rate: float,
                                learning_rate_function: Callable[[float, float, int], float],
                                initial_value_learning_rate: float,
                                attenuation_parameter: float,
                                batch_size: int = 1,
                                eps: float = 1e-6,
                                ):
    data_size, number_coefficients = X.shape
    W = np.zeros(number_coefficients)

    Q_last = sum(((Y[i] - np.dot(W, X[i])) ** 2) / data_size for i in range(data_size))
    Q_new = Q_last

    h = initial_value_learning_rate
    cnt_epoch = 0
    cnt_iter = 0
    while True:
        list_indexes = list(range(0, data_size))

        for i in range(math.ceil(data_size / batch_size)):
            random_indexes = random.sample(list_indexes,
                                           k=min(len(list_indexes), batch_size))
            list_indexes = list(filter(lambda x: x not in set(random_indexes), list_indexes))

            gradient = sum((2 * X[index] * (X[index].dot(W) - Y[index])) / len(random_indexes) for index in random_indexes)
            W = W - h * gradient
            cnt_iter += 1
            h = learning_rate_function(h, attenuation_parameter, cnt_iter)

            current_error = (Y[i] - np.dot(W, X[i])) ** 2
            Q_new = forgetting_rate * current_error + (1 - forgetting_rate) * Q_new

        if abs(Q_new - Q_last) < eps:
            break

        Q_last = Q_new
        cnt_epoch += 1

    return [cnt_epoch, cnt_iter, W, Q_new]

def constant_learning_rate_function(initial_value_learning_rate: float, attenuation_parameter: float, n: int) -> float:
  return initial_value_learning_rate

def time_learning_rate_function(initial_value_learning_rate: float, attenuation_parameter: float, n: int) -> float:
  return initial_value_learning_rate / (1 + attenuation_parameter * n)

def exponential_learning_rate_function(initial_value_learning_rate: float, attenuation_parameter: float, n: int) -> float:
  return initial_value_learning_rate * math.exp(-attenuation_parameter)


def stochastic_gradient_descent_with_constant_learning_rate(X: np.array,
                                Y: np.array,
                                batch_size: int = 1,
                                learning_rate: float = 0.01,
                                forgetting_rate: float = 0.1,
                                eps: float = 1e-6):
  return stochastic_gradient_descent(X, Y, forgetting_rate, constant_learning_rate_function, learning_rate, 1, batch_size, eps)

def stochastic_gradient_descent_with_time_learning_rate(X: np.array,
                                Y: np.array,
                                batch_size: int = 1,
                                initial_value_learning_rate: float = 0.1,
                                attenuation_parameter: float = 0.5,
                                forgetting_rate: float = 0.1,
                                eps: float = 1e-6):
  return stochastic_gradient_descent(X, Y, forgetting_rate, time_learning_rate_function, initial_value_learning_rate, attenuation_parameter, batch_size, eps)

def stochastic_gradient_descent_with_exponential_learning_rate(X: np.array,
                                Y: np.array,
                                batch_size: int = 1,
                                initial_value_learning_rate: float = 0.1,
                                attenuation_parameter: float = 0.5,
                                forgetting_rate: float = 0.1,
                                eps: float = 1e-6):
  return stochastic_gradient_descent(X, Y, forgetting_rate, exponential_learning_rate_function, initial_value_learning_rate, attenuation_parameter, batch_size, eps)


**Задача:**    
Реализовать и исследовать на эффективность SGD для полиномиальной
регрессии с добавлением регуляризации в модель разных методов
регуляризации (L1, L2, Elastic регуляризации).

**О регуляризации:**   
Один из способов бороться с негативным эффектом излишнего подстраивания под данные — использование регуляризации, т. е. добавление некоторого штрафа за большие значения коэффициентов в модели. Тем самым запрещаются слишком "резкие" изгибы, и предотвращается переобучение.
1. L1-регуляризация    
$R(w) = \sum_{i=1}^N |b_i| $
2. L2-регуляризация    
$R(w) = \sum_{i=1}^N (b_i)^2 $
3. Elastic регуляризация    
$R(w) = \lambda_1 \cdot \sum_{i=1}^N |b_i| + \lambda_2 \cdot \sum_{i=1}^N (b_i)^2 $

**Условия реализации полиномиальной регрессии:**
1. $y$ может зависеть от каждой независимой переменной $x_k$ в виде степени, значение которой может быть от 1 до N
2. N = 9
3. Понятно, что на одинаковых входных данных разные функции $g1$ и $g2$ могут давать равные с некоторой точностью ответы. Так что написанный метод возвращает три ответа, у которых наименьшее значение функции ошибки.



In [25]:
# Доп 1 - полиномы

def polynom_stochastic_gradient_descent(X: np.array,
                                Y: np.array,
                                forgetting_rate: float = 0.1,
                                learning_rate: float = 0.01,
                                batch_size: int = 1,
                                eps: float = 1e-6,
                                ):

    data_size, N = X.shape

    result_W = np.array([np.zeros(N), np.zeros(N), np.zeros(N)])
    result_D = np.zeros(3)
    result_cnt_epoch = np.zeros(3)
    result_cnt_iter = np.zeros(3)
    result_Q = np.array([10e9, 10e9, 10e9])

    for permutation in itertools.product('123456789', repeat=N):
      D = ''.join(permutation)

      W = np.zeros(N)

      Q_last = sum((Y[i] - sum(W[j] * pow(X[i][j], int(D[j])) for j in range(N))) ** 2 / data_size for i in range(data_size))
      Q_new = Q_last

      cnt_epoch = 0
      cnt_iter = 0

      while True:
        list_indexes = list(range(0, data_size))

        for i in range(math.ceil(data_size / batch_size)):
            random_indexes = random.sample(list_indexes,
                                           k=min(len(list_indexes), batch_size))
            list_indexes = list(filter(lambda x: x not in set(random_indexes), list_indexes))

            gradient = sum((2 * X[index] * (sum(W[j] * pow(X[index][j], int(D[j])) for j in range(N)) - Y[index])) / len(random_indexes) for index in random_indexes)
            W = W - learning_rate * gradient
            cnt_iter += 1

            current_error = (Y[i] - sum(W[j] * pow(X[i][j], int(D[j])) for j in range(N))) ** 2
            Q_new = forgetting_rate * current_error + (1 - forgetting_rate) * Q_new

        if abs(Q_new - Q_last) < eps:
            break

        Q_last = Q_new
        cnt_epoch += 1

      max_Q = result_Q[0]
      k_max_Q = 0
      for k in range(3):
        if result_Q[k] > max_Q:
          max_Q = result_Q[k]
          k_max_Q = k

      if (Q_new < max_Q):
        print(D)
        result_W[k_max_Q] = W
        result_D[k_max_Q] = int(D)
        result_cnt_epoch[k_max_Q] = cnt_epoch
        result_cnt_iter[k_max_Q] = cnt_iter
        result_Q[k_max_Q] = Q_new

    return [result_cnt_epoch, result_cnt_iter, result_W, result_D, result_Q]


# Доп1 - регуляризация

def stochastic_gradient_descent_with_regularization(X: np.array,
                                Y: np.array,
                                regularization: Callable[[np.array], float],
                                forgetting_rate: float = 0.1,
                                learning_rate: float = 0.01,
                                batch_size: int = 1,
                                eps: float = 1e-6,
                                ):
    data_size, number_coefficients = X.shape
    W = np.zeros(number_coefficients)
    #W = np.full(shape=number_coefficients, fill_value=500)

    Q_last = sum(((Y[i] - np.dot(W, X[i])) ** 2) / data_size for i in range(data_size))
    Q_new = Q_last

    cnt_epoch = 0
    cnt_iter = 0
    while True:
        list_indexes = list(range(0, data_size))

        for i in range(math.ceil(data_size / batch_size)):
            random_indexes = random.sample(list_indexes,
                                           k=min(len(list_indexes), batch_size))
            list_indexes = list(filter(lambda x: x not in set(random_indexes), list_indexes))

            gradient = sum((2 * X[index] * (X[index].dot(W) - Y[index])) / len(random_indexes) for index in random_indexes)
            W = W - learning_rate * gradient
            cnt_iter += 1

            current_error = (Y[i] - np.dot(W, X[i])) ** 2 + regularization(W)
            Q_new = forgetting_rate * current_error + (1 - forgetting_rate) * Q_new

        if abs(Q_new - Q_last) < eps:
            break

        Q_last = Q_new
        cnt_epoch += 1

    return [cnt_epoch, cnt_iter, W, Q_new]

def L1(W: np.array) -> float:
  return sum(abs(W[i]) for i in range(len(W)))

def L2(W: np.array) -> float:
  return sum(pow(W[i], 2) for i in range(len(W)))

def Elastic(W: np.array) -> float:
  return sum(0.5 * abs(W[i]) + 0.5 * pow(W[i], 2) for i in range(len(W)))

# Тесты

In [26]:
# Тесты

# Для размеров батчей (learning_rate = 0.01, forgetting_rate = 0.1)

def print_result_of_batch_test(batch_size, data):
  theta = stochastic_gradient_descent_with_constant_learning_rate(data[0], data[1])

  print("Batch-size = ", batch_size, ":")
  print("Count_epoch : ", theta[0], " count_iterations : ", theta[1])
  print("Result : ", theta[2])

def one_batch_test(input_coefficients, count_data, batch_size1, batch_size2, batch_size3):
  data = data_fabric(count_data, input_coefficients, np.ones((len(input_coefficients))))
  print("Count_data : ", count_data, "input_coefficients : ", input_coefficients)
  print_result_of_batch_test(batch_size1, data)
  print_result_of_batch_test(batch_size2, data)
  print_result_of_batch_test(batch_size3, data)
  print("----------------------------------------------")
  print()

def test_batches_standart_SGD():
  random_input_coefficients1 = np.array(list(np.random.randint(-10, 10, 6)))
  one_batch_test(random_input_coefficients1, 10, 1, 3, 6)

  random_input_coefficients2 = np.array(list(np.random.randint(-10, 10, 6)))
  one_batch_test(random_input_coefficients2, 100, 1, 3, 6)

  many_input_coefficients = [i for i in range(100)]
  one_batch_test(many_input_coefficients, 50, 1, 50, 100)

# Для разных функций изменения шага

def print_result_of_test(theta):
  print("Count_epoch : ", theta[0], " count_iterations : ", theta[1])
  print("Result : ", theta[2])

def test_constant_learning_rate(data, learning_rate: np.array):
  for i in range(len(learning_rate)):
    print()
    coefficient = learning_rate[i]
    theta = stochastic_gradient_descent_with_constant_learning_rate(data[0], data[1], learning_rate=coefficient)
    print("initial_value_learning_rate = ", coefficient)
    print_result_of_test(theta)
  print("----------------------------------------------------------------------------------------------------")
  print()

def test_NON_constant_learning_rate(data, SGD_function, initial_value_learning_rate: np.array, attenuation_parameter: np.array):
  for i in range(len(initial_value_learning_rate)):
    for j in range(len(attenuation_parameter)):
      coefficient = initial_value_learning_rate[i]
      parameter = attenuation_parameter[j]
      print()
      print("initial_value_learning_rate = ", coefficient, " attenuation_parameter = ", parameter)
      theta = SGD_function(data[0], data[1], initial_value_learning_rate=coefficient, attenuation_parameter=parameter)
      print_result_of_test(theta)
  print("----------------------------------------------------------------------------------------------------")
  print()

def test_learning_rate():
  input_coefficients = np.array(list(np.random.randint(-10, 10, 10)))
  print("input_coefficients : ", input_coefficients)
  print("----------------------------------------------------------------------------------------------------")
  print()
  data = data_fabric(20, input_coefficients, np.ones((len(input_coefficients))))

  print("Constant learning rate")
  test_constant_learning_rate(data, np.array([0.01, 0.05, 0.1]))

  print("Time learning rate")
  test_NON_constant_learning_rate(data, stochastic_gradient_descent_with_time_learning_rate, np.array([1, 0.2]), np.array([1, 0.1, 0.01]))

  print("Exponential learning rate")
  test_NON_constant_learning_rate(data, stochastic_gradient_descent_with_exponential_learning_rate, np.array([1, 0.2]), np.array([1, 0.1, 0.01]))

# Тест для полинома

def polynom_test():
  data = data_fabric(20, np.array([142, 201, -56]), np.array([9, 2, 4]))

  R = polynom_stochastic_gradient_descent(data[0], data[1], eps=1e-2)
  for k in range(3):
    print("Coefficients : ", R[2][k], " degrees : [", ' '.join(str(int(R[3][k]))), "] error : ", R[4][k])

# Тест для регуляризации

def regularization_test():
  input_coefficients = list([0, 0, 0, 5, 0, 0, 7, 0, 0, 0])
  #np.array(list(np.random.randint(-10, 10, 10)))
  print("input_coefficients : ", input_coefficients)
  print()
  data = data_fabric(20, input_coefficients, np.ones((len(input_coefficients))))

  theta = stochastic_gradient_descent_with_constant_learning_rate(data[0], data[1])
  print("Default")
  print_result_of_test(theta)
  theta_l1 = stochastic_gradient_descent_with_regularization(data[0], data[1], L1)
  print()
  print("L1")
  print_result_of_test(theta_l1)
  theta_l2 = stochastic_gradient_descent_with_regularization(data[0], data[1], L2)
  print()
  print("L2")
  print_result_of_test(theta_l2)
  theta_elastic = stochastic_gradient_descent_with_regularization(data[0], data[1], Elastic)
  print()
  print("Elastic")
  print_result_of_test(theta_elastic)

# Для сравнения с библиотечными реализациями
last_epoch = 0

def test_library(count_coefficient=10, count_data=100, batch_size=32, eps1=1e-7):
    global last_epoch

    class LossCallback(Callback):
        eps = eps1
        prev = 0.0

        def on_batch_end(self, batch, logs=None):
            if logs['loss'] is None:
                return
            elif abs(self.prev - logs['loss']) < self.eps:

                self.model.stop_training = True
            self.prev = logs['loss']

        def on_epoch_end(self, epoch, logs=None):
            global last_epoch
            last_epoch = epoch

    optimizers = {
        "adagrad": tf.keras.optimizers.Adagrad(learning_rate=0.8, initial_accumulator_value=0.05),
        "nesterov": tf.keras.optimizers.SGD(learning_rate=0.07, nesterov=True, momentum=0.6),
        "sgd": tf.keras.optimizers.SGD(learning_rate=0.08),
        "rmsprop": tf.keras.optimizers.RMSprop(learning_rate=0.08, rho=0.2, momentum=0.1),
        "adam": tf.keras.optimizers.Adam(learning_rate=0.07, beta_1=0.9, beta_2=0.99),
        "momentum": tf.keras.optimizers.SGD(learning_rate=0.07, momentum=0.4),
    }
    input_coefficients = np.array(list(np.random.randint(-30, 30, count_coefficient)))
    print("input_coefficients : ", input_coefficients)
    print("----------------------------------------------------------------------------------------------------")
    data = data_fabric(count_data, input_coefficients, np.ones((len(input_coefficients))))
    X, y = data[0], data[1]

    for method in optimizers.keys():
        print(method)
        model = Sequential([
            Dense(1)
        ])

        model.compile(optimizer=optimizers[method], loss='mean_squared_error')
        model.fit(X, y, batch_size=batch_size, epochs=10000, verbose=0,
                  callbacks=[LossCallback()])

        print("[", end="")
        for i in range(count_coefficient):
            print(model.layers[0].weights[0][i][0].numpy(), end=", ")
        print("]")
        print(f"Epochs: {last_epoch}, iterations: {last_epoch * batch_size + 1}")
        last_epoch = 0


# Main

#if __name__ == '__main__':
  #regularization_test()


In [27]:
test_batches_standart_SGD()

Count_data :  10 input_coefficients :  [-2 -9 -4 -7 -3  7]
Batch-size =  1 :
Count_epoch :  366  count_iterations :  3670
Result :  [-2.08594145 -8.57050025 -4.47411671 -6.92920355 -2.74022366  6.8019926 ]
Batch-size =  3 :
Count_epoch :  518  count_iterations :  5190
Result :  [-2.04245383 -8.80946979 -4.20323987 -6.97816654 -2.89603962  6.92679078]
Batch-size =  6 :
Count_epoch :  599  count_iterations :  6000
Result :  [-2.02714258 -8.87789528 -4.12836313 -6.98699715 -2.93485138  6.95575355]
----------------------------------------------

Count_data :  100 input_coefficients :  [ 7  2 -1 -8  7  3]
Batch-size =  1 :
Count_epoch :  55  count_iterations :  5600
Result :  [ 7.00248662  2.0026859  -0.99729798 -7.99674867  6.99325057  2.99536441]
Batch-size =  3 :
Count_epoch :  55  count_iterations :  5600
Result :  [ 7.00263105  2.00274245 -0.9972052  -7.99663846  6.99316118  2.99533395]
Batch-size =  6 :
Count_epoch :  55  count_iterations :  5600
Result :  [ 7.00258238  2.00277152 -0.

In [28]:
test_learning_rate()

input_coefficients :  [-2  5  1  4  7 -2 -4  0 -4 -7]
----------------------------------------------------------------------------------------------------

Constant learning rate

initial_value_learning_rate =  0.01
Count_epoch :  408  count_iterations :  8180
Result :  [-1.89632457  4.75572201  1.00723259  4.14238938  7.04318151 -2.22887587
 -3.96507664 -0.17455826 -3.79369482 -6.87943426]

initial_value_learning_rate =  0.05
Count_epoch :  109  count_iterations :  2200
Result :  [-1.9540273   4.89932037  0.99536601  4.06574658  7.0200911  -2.09486572
 -3.98633306 -0.0659529  -3.91511394 -6.94654103]

initial_value_learning_rate =  0.1
Count_epoch :  91  count_iterations :  1840
Result :  [-1.99160545  4.98262079  0.99908705  4.00887318  7.00298209 -2.01447033
 -3.99903665 -0.01221307 -3.98637984 -6.99228384]
----------------------------------------------------------------------------------------------------

Time learning rate

initial_value_learning_rate =  1.0  attenuation_paramete

In [29]:
regularization_test()

input_coefficients :  [0, 0, 0, 5, 0, 0, 7, 0, 0, 0]

Default
Count_epoch :  221  count_iterations :  4440
Result :  [ 5.72533692e-02  2.22549572e-03  8.70864097e-03  4.97960038e+00
  5.67219810e-02 -2.92900281e-02  6.88218394e+00 -5.88034175e-04
  3.10023323e-02  1.62649291e-02]

L1
Count_epoch :  512  count_iterations :  10260
Result :  [-7.59426148e-04  3.90292901e-03  2.19467799e-04  5.00411701e+00
  2.48152179e-03 -3.07222173e-03  6.99594133e+00 -4.90567966e-03
  3.92938420e-04 -7.68869836e-06]

L2
Count_epoch :  771  count_iterations :  15440
Result :  [-4.19049433e-04  1.04807141e-03  7.34706078e-05  5.00106614e+00
  5.33763185e-04 -5.46902261e-04  6.99919280e+00 -1.14937835e-03
 -4.44698338e-05 -1.50027871e-04]

Elastic
Count_epoch :  717  count_iterations :  14360
Result :  [-5.49871005e-04  1.41568518e-03  9.77262176e-05  5.00146595e+00
  7.37059522e-04 -7.67097179e-04  6.99888238e+00 -1.57635441e-03
 -3.89289724e-05 -1.93538688e-04]
